In [1]:
import tensorflow as tf
import numpy as np
import timeit
from sklearn.utils import shuffle
from load_cifar import *
import time
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

<h1>Extract MNIST data</h1>

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
# get mnist data, with one_hot encoding, reshape = False (that means images are not flatten)
mnist = input_data.read_data_sets("MNIST_data/",reshape=False,one_hot=True)
# suppress warnings
tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<h1>Prepare training, validation and testing data</h1>

In [3]:
x_train, y_train = mnist.train.images, mnist.train.labels
x_validation, y_validation = mnist.validation.images, mnist.validation.labels
x_test, y_test = mnist.test.images, mnist.test.labels

# pad images with 0s (28x28 to 32x32)
x_train_pad = np.pad(x_train, ((0,0), (2,2), (2,2), (0,0)), mode='constant', constant_values=0)
x_validation_pad = np.pad(x_validation, ((0,0), (2,2), (2,2), (0,0)), mode='constant', constant_values=0)
x_test_pad = np.pad(x_test, ((0,0), (2,2), (2,2), (0,0)), mode='constant', constant_values=0)

<h1>Define hyperparameter</h1>

In [4]:
lr = 0.0007
batch_size = 128
num_epochs = 20
num_classes = 10

In [5]:
tf.reset_default_graph()

<h1>Placeholder</h1>

In [6]:
X = tf.placeholder(tf.float32,[None,32,32,1],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')
Training = tf.placeholder(tf.bool, None)

<h1>Define LeNet-5</h1>

In [7]:
C1 = tf.layers.conv2d(X, filters=6, kernel_size=5, activation=tf.nn.relu)

S2 = tf.layers.max_pooling2d(C1, pool_size=2, strides=2)

S2_bn = tf.layers.batch_normalization(S2)

# S2_bn = tf.nn.relu(S2_bn)

C3 = tf.layers.conv2d(S2_bn, filters=16, kernel_size=5, activation=tf.nn.relu)

S4 = tf.layers.average_pooling2d(C3, pool_size=2, strides=2)

S4_bn = tf.layers.batch_normalization(S4)

# S4_bn = tf.nn.relu(S4_bn)

# F5 = tf.layers.flatten(tf.layers.conv2d(S4, filters=120, kernel_size=5, activation=tf.nn.relu))

F5 = tf.layers.dense(tf.layers.flatten(S4_bn), units=120, activation=tf.nn.relu)

# F5 = tf.layers.dropout(F5, rate=0.6, training=Training)

F5_bn = tf.layers.batch_normalization(F5)

F6 = tf.layers.dense(F5_bn, units=84, activation=tf.nn.relu)

F6 = tf.layers.dropout(F6, rate=0.6, training=Training)

F6_bn = tf.layers.batch_normalization(F6)

logits = tf.layers.dense(F6_bn, units=10)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<h1>Cost and optimization</h1>

In [8]:
# define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')

# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

# compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(tf.nn.softmax(logits),1),tf.argmax(Y,1))

# compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

<h1>Training, validating, testing</h1>
<h2>1. Print out validation accuracy after each training epoch</h2>
<h2>2. Print out training time on each epoch</h2>
<h2>3. Print out testing accuracy</h2>

In [9]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(num_epochs):
        s = time.time()
        data_batch = mini_batch(x_train_pad, y_train, batch_size)
        for x, y in data_batch:
            sess.run(train_op, feed_dict = {X: x, Y: y, Training: True})
        validation_acc = sess.run(accuracy, feed_dict = {X:x_validation_pad, Y:y_validation, Training: False})
        f = time.time()
        print('Epoch:' + str(i) + '   Time:{:.3f}'.format(f - s) + '   Current validation accuracy: ' + str(validation_acc))
    test_acc = sess.run(accuracy, feed_dict = {X:x_test_pad, Y:y_test, Training: False})
    print('Test accuracy: ')
    print(test_acc)

Epoch:0   Time:3.324   Current validation accuracy: 0.9572
Epoch:1   Time:1.682   Current validation accuracy: 0.9732
Epoch:2   Time:1.669   Current validation accuracy: 0.9794
Epoch:3   Time:1.677   Current validation accuracy: 0.9812
Epoch:4   Time:1.659   Current validation accuracy: 0.985
Epoch:5   Time:1.680   Current validation accuracy: 0.9868
Epoch:6   Time:1.656   Current validation accuracy: 0.9862
Epoch:7   Time:1.651   Current validation accuracy: 0.9872
Epoch:8   Time:1.649   Current validation accuracy: 0.9886
Epoch:9   Time:1.652   Current validation accuracy: 0.9888
Epoch:10   Time:1.651   Current validation accuracy: 0.9898
Epoch:11   Time:1.649   Current validation accuracy: 0.9898
Epoch:12   Time:1.650   Current validation accuracy: 0.9904
Epoch:13   Time:1.652   Current validation accuracy: 0.99
Epoch:14   Time:1.646   Current validation accuracy: 0.99
Epoch:15   Time:1.662   Current validation accuracy: 0.9908
Epoch:16   Time:1.655   Current validation accuracy: 0.